In [19]:
import traitlets
import os
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [2]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

In [10]:
camera = Camera.instance(width=224, height=224)

In [21]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='steering')

def display_steering(camera_image):
    image = np.copy(camera_image)
    steering = steering_slider.value
    x = int(steering * 224 / 2 + 112)
    y = int(224 - 0.15 * 224)
    image = cv2.circle(image, (x, y), 3, (0, 255, 0), 2)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

camera.unobserve_all()
time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_steering)

display(widgets.HBox([image_widget, target_widget]), steering_slider)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0, step=0.001)

In [22]:
controller = widgets.Controller(index=0)

display(controller)

Controller()

In [23]:
widgets.jsdlink((controller.axes[2], 'value'), (steering_slider, 'value'))

# Collect steering data

In [24]:
def steering_uuid(steering):
    value = steering * 50 + 50
    return 'steering_%03d_%s' % (value, uuid1())

In [29]:
DATASET_DIR = 'dataset_all'
#DATASET_DIR = 'dataset_left'
#DATASET_DIR = 'dataset_forward'
#DATASET_DIR = 'dataset_right'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def save_steering(change):
    if change['new']:
        uuid = steering_uuid(steering_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[13].observe(save_steering, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

Directories not created becasue they already exist


In [33]:
save_steering({'new': True})

In [ ]:
!zip -r -q lego_city_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}